In [13]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
API_KEY = os.getenv('API_KEY')

In [19]:
import pandas as pd

df = pd.read_csv('acm.csv')

'HAID: Human-AI Interaction for Dementia Care'

In [75]:
from pydantic import BaseModel
class AbstractSummary(BaseModel):
    class Summary(BaseModel):
        bullet: str
        effect: str
        
    keywords: list[str]
    summaries: list[Summary]

In [115]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=API_KEY,
)

prompt = """Extract key findings from the abstract of this paper in bullet points focusing on the effect of AI,
emphasizing complete and clear statements of results or conclusions (e.g., \"AI Chatbot can induce creativity in brainstorming session\" or \"AI-enabled knowledge graph generation can be used for taxonomical tasks\").
Avoid including any context or procedural details (e.g. the paper hosts workshops sessions to test the hypothesis).
Each of the statement should be self-explanatory, independent from other bullets, and informative.
Terms like higher than (...), control condition, must be elaborated and explained or avoided.
Superlative terms, e.g., strongest or highest, must be modified relatively to other findings or avoid the terms.
Number of bullets should be as small as possible, corresponding to the number of significant, statistical, and practical findings.

The format of the output is as follows:
summary.bullet -> summary
summary.effect -> "Positive" | "Negative" | "Neutral" analyzing the effect of AI on the findings on the society and human
keywords -> topics and themes of the paper. With these papers being about AI and HCI, terms related to the two, e.g. "AI application", "Human-centered design", "Human-AI interaction", must be omitted. The included terms should be around the thematic concerns of the paper, e.g. "Medical", "Creativity", "Productivity".
"""

def get_summary(abstract):
    chat_completion = client.beta.chat.completions.parse(
        messages=[
            {
                "role": "user",
                "content": prompt + " The following is the abstract: " + abstract,
            },
        ],
        model="gpt-4o",
        response_format=AbstractSummary,
    )
    
    return chat_completion.choices[0].message.parsed

def print_summary(summary):
    print(f"Keywords: {summary.keywords}")
    for s in summary.summaries:
        print(f"- {s.bullet} | Effect: {s.effect}")

In [116]:
for i in range(4):
    print(df.iloc[i]['title'])
    s = get_summary(df.iloc[i]['abstract'])
    print_summary(s)
    print()

HAID: Human-AI Interaction for Dementia Care
Keywords: ['Dementia', 'Caregiving', 'Quality of Life']
- AI technologies can improve the quality of life for individuals with dementia. | Effect: Positive
- AI-driven platforms support caregivers in managing dementia care challenges. | Effect: Positive

Theory of Mind in Human-AI Interaction
Keywords: ['Theory of Mind', 'Social Interaction', 'Mental States Attribution', 'Human Expectations']
- AI systems with human-like Theory of Mind capabilities can potentially interpret and attribute mental states such as intentions, emotions, and beliefs effectively. | Effect: Positive
- People may naturally attribute mental states like blame, emotions, and intentions to AI, influencing how the AI should behave to meet human expectations. | Effect: Positive
- Mutual Theory of Mind in AI is emerging as a new paradigm that allows both human and AI to understand and attribute mental states to each other. | Effect: Positive

Assessing Human-AI Interaction E

In [117]:
abs = "AI is increasingly used to enhance images and videos, both intentionally and unintentionally. As AI editing tools become more integrated into smartphones, users can modify or animate photos into realistic videos. This study examines the impact of AI-altered visuals on false memories--recollections of events that didn't occur or deviate from reality. In a pre-registered study, 200 participants were divided into four conditions of 50 each. Participants viewed original images, completed a filler task, then saw stimuli corresponding to their assigned condition: unedited images, AI-edited images, AI-generated videos, or AI-generated videos of AI-edited images. AI-edited visuals significantly increased false recollections, with AI-generated videos of AI-edited images having the strongest effect (2.05x compared to control). Confidence in false memories was also highest for this condition (1.19x compared to control). We discuss potential applications in HCI, such as therapeutic memory reframing, and challenges in ethical, legal, political, and societal domains."
s = get_summary(abs)

In [118]:
print_summary(s)

Keywords: ['False Memories', 'Ethical Concerns', 'Therapeutic Applications']
- AI-edited visuals significantly increased the occurrence of false memories among participants. | Effect: Negative
- AI-generated videos of AI-edited images resulted in the highest increase in false memory confidence among participants. | Effect: Negative
- AI tools used in altering visuals can be applied for therapeutic memory reframing objectives. | Effect: Positive
